In [1]:
from pydrake.all import (RobotDiagramBuilder, 
                         VisualizationConfig,
                         RigidTransform,
                         Rgba,
                         StartMeshcat, 
                         ApplyVisualizationConfig)
from cspace_utils.plotting import plot_triad
import numpy as np
import ipywidgets as widgets
from functools import partial
from utils import DEVEL_PATH
meshcat = StartMeshcat()
path = DEVEL_PATH +"/movable_capsule.urdf"
rbuilder = RobotDiagramBuilder()
plant= rbuilder.plant()
builder = rbuilder.builder()
parser = rbuilder.parser()
scene_graph = rbuilder.scene_graph()
parser.AddModels(path)
config = VisualizationConfig()
config.enable_alpha_sliders = False
config.publish_contacts=False
config.publish_inertia = False
config.default_proximity_color = Rgba(0.8,0,0,0.5)
#AddDefaultVisualization(builder.builder(), meshcat, enable_alpha_sliders=True)
plant.Finalize()
ApplyVisualizationConfig(config, builder, meshcat=meshcat)
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(diagram_context)
diagram.ForcedPublish(diagram_context)

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [2]:
q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.001, description=f"q{i}"))

def handle_slider_change(change, idx):

    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    if query.HasCollisions():
        print("col")
    # for i, rt in enumerate(rts_):
    #     plot_triad(rt, meshcat, f"full/link_frame_{i}")
    # for i in range(mkt.num_links):
    #     rt = RigidTransform(transforms_minimal[4*i:4*(i+1),:])
    #     plot_triad(rt, meshcat, f"minimal/link_frame_{i}")
    diagram.ForcedPublish(diagram_context)

idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=0.693, min=-0.693, step=0.001)

FloatSlider(value=0.0, description='q1', max=0.693, min=-0.693, step=0.001)

FloatSlider(value=0.0, description='q2', max=0.693, min=-0.099, step=0.001)

FloatSlider(value=0.0, description='q3', max=3.1086, min=-3.1086, step=0.001)

FloatSlider(value=0.0, description='q4', max=3.1086, min=-3.1086, step=0.001)

FloatSlider(value=0.0, description='q5', max=3.1086, min=-3.1086, step=0.001)

In [3]:
def print_conf(q):
    print(f"np.array([{q[0]:.3f}, {q[1]:.3f}, {q[2]:.3f}, {q[3]:.3f}, {q[4]:.3f}, {q[5]:.3f}])")

In [4]:
print_conf(q)

np.array([0.000, 0.000, 0.000, 0.000, 0.000, 0.000])


In [5]:
col_free = [
    np.array([0.2, 0.1, -0.0, 0.5, -0.3, 0.5]),
    np.array([0.2, 0.1, -0.0, 0.5, 1.9, 0.5]),
    np.array([-0.2, -0.0, -0.0, 0.5, 1.9, 0.5]),
    np.array([-0.2, -0.0, -0.0, 0.5, 3.1, 0.5])
]

col = [
    np.array([-0.5, -0.0, 0.1, 0.0, 0.7, 0.5]),
    np.array([0.5, -0.1, 0.1, 0.0, 0.7, -0.8]),
    np.array([0.5, 0.5, 0.2, 0.0, 0.7, -0.8]),
    np.array([0.1, 0.0, 0.4, -0.3, 0.1, -1.1])
]

In [8]:
plant.SetPositions(plant_context, col[1])
diagram.ForcedPublish(diagram_context)
plant.SetPositions(plant_context, q)
query = plant.get_geometry_query_input_port().Eval(plant_context)
if query.HasCollisions():
    print("col")

In [7]:
p1 = np.array([0.468, -0.06288, 0.147])
p2 = np.array([0.3848,-0.3047, 0.2819])

from cspace_utils.plotting import plot_edge
from pydrake.all import Rgba
plot_edge(meshcat, p1, p2, 'shortest', Rgba(0,0,0,1), size = 0.01)

In [9]:
np.linalg.norm(p2-p1)

0.28913173883197263